In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

2024-09-13 23:05:47.293183: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-13 23:05:47.329496: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-13 23:05:47.329533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-13 23:05:47.330694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-13 23:05:47.340987: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-09-13 23:05:48.419162: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-13 23:05:48.485966: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-13 23:05:48.486019: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


2024-09-13 23:05:48.504479: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-13 23:05:48.504582: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-13 23:05:48.504600: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-13 23:05:48.602142: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-13 23:05:48.602206: I external/local_xla/xla/stream_executor

In [4]:
tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
def use_tokenizer_training(data):
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(data)
    X_sequences = tokenizer.texts_to_sequences(data)
    X_padded = pad_sequences(X_sequences, padding='post')
    return tokenizer,X_sequences,X_padded

In [7]:
def use_label_tokenizer_training(data):
    label_tokenizer = tf.keras.preprocessing.text.Tokenizer()
    label_tokenizer.fit_on_texts([label for label in data])
    y_sequences = label_tokenizer.texts_to_sequences(data)
    y_padded = pad_sequences(y_sequences,padding='post')
    return label_tokenizer,y_sequences,y_padded

In [8]:
df = pd.read_json("english_balanced_10k.jsonl",lines=True)

In [9]:
df = df[['tokenised_unmasked_text','token_entity_labels']]

In [10]:
df.rename({"token_entity_labels":"target","tokenised_unmasked_text":"text"},axis=1,inplace=True)

In [11]:
x = df['text']
y = df['target']

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
tokenizer,x_train_seq,x_train_padded = use_tokenizer_training(x)
label_tokenizer,y_train_seq,y_train_padded = use_label_tokenizer_training(y)

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x_train_padded,y_train_padded,random_state=123)

In [15]:
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,random_state=123)

In [31]:
original_dict = label_tokenizer.word_index
idx2tag = {v: k for k, v in original_dict.items()}
idx2tag[0]=0
print(idx2tag)

{1: 'o', 2: 'i-useragent', 3: 'i-ipv6', 4: 'i-ethereumaddress', 5: 'i-bitcoinaddress', 6: 'i-ip', 7: 'i-email', 8: 'i-url', 9: 'i-litecoinaddress', 10: 'i-iban', 11: 'i-mac', 12: 'i-phone_number', 13: 'i-date', 14: 'i-phoneimei', 15: 'i-creditcardnumber', 16: 'i-password', 17: 'i-vehiclevin', 18: 'i-ipv4', 19: 'i-maskednumber', 20: 'i-company_name', 21: 'i-username', 22: 'i-accountnumber', 23: 'i-streetaddress', 24: 'i-ssn', 25: 'i-jobtitle', 26: 'i-amount', 27: 'i-zipcode', 28: 'i-bic', 29: 'i-vehiclevrm', 30: 'i-lastname', 31: 'i-currency', 32: 'b-firstname', 33: 'i-time', 34: 'i-city', 35: 'b-lastname', 36: 'b-date', 37: 'b-prefix', 38: 'b-email', 39: 'i-street', 40: 'i-secondaryaddress', 41: 'i-accountname', 42: 'b-jobtitle', 43: 'b-username', 44: 'i-firstname', 45: 'i-prefix', 46: 'b-url', 47: 'b-time', 48: 'b-jobarea', 49: 'b-city', 50: 'b-company_name', 51: 'b-amount', 52: 'b-middlename', 53: 'b-ipv4', 54: 'b-currency', 55: 'b-accountnumber', 56: 'i-buildingnumber', 57: 'b-phone

In [17]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
max_len = x_train_padded.shape[1]
num_tags = len(label_tokenizer.word_index) + 1

In [18]:
from tensorflow.keras.layers import Embedding,Dense,LSTM,Bidirectional,TimeDistributed
from tensorflow.keras.models import Sequential

In [19]:
model = Sequential()

In [20]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(units=256, return_sequences=True)))
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(TimeDistributed(Dense(128, activation='relu')))
model.add(Dense(num_tags,activation = 'softmax'))

2024-09-13 23:05:49.772797: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-09-13 23:05:49.772833: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at ptxas
2024-09-13 23:05:49.772863: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 214, 100)          1142100   
                                                                 
 bidirectional (Bidirection  (None, 214, 512)          731136    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 214, 256)          656384    
 onal)                                                           
                                                                 
 bidirectional_2 (Bidirecti  (None, 214, 128)          164352    
 onal)                                                           
                                                                 
 time_distributed (TimeDist  (None, 214, 128)          16512     
 ributed)                                             

In [23]:
history = model.fit(
    x_train, y_train,                
    batch_size=32,                   
    epochs=15,                       
    validation_data=(x_val, y_val)
)

Epoch 1/15


2024-09-13 23:05:54.096584: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-09-13 23:05:54.141306: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-09-13 23:05:54.145241: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-09-13 23:05:54.151051: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-09-13 23:05:54.153706: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-09-13 23:05:54.153797: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kern

192/192 [==============================] - 28s 124ms/step - loss: 0.7233 - accuracy: 0.8716 - val_loss: 0.4115 - val_accuracy: 0.9155
Epoch 2/15
192/192 [==============================] - 17s 87ms/step - loss: 0.3405 - accuracy: 0.9209 - val_loss: 0.2950 - val_accuracy: 0.9242
Epoch 3/15
192/192 [==============================] - 16s 83ms/step - loss: 0.2515 - accuracy: 0.9339 - val_loss: 0.2282 - val_accuracy: 0.9388
Epoch 4/15
192/192 [==============================] - 15s 81ms/step - loss: 0.2009 - accuracy: 0.9436 - val_loss: 0.2061 - val_accuracy: 0.9477
Epoch 5/15
192/192 [==============================] - 15s 78ms/step - loss: 0.1667 - accuracy: 0.9525 - val_loss: 0.1600 - val_accuracy: 0.9534
Epoch 6/15
192/192 [==============================] - 15s 77ms/step - loss: 0.1348 - accuracy: 0.9596 - val_loss: 0.1343 - val_accuracy: 0.9602
Epoch 7/15
192/192 [==============================] - 15s 78ms/step - loss: 0.1113 - accuracy: 0.9659 - val_loss: 0.1134 - val_accuracy: 0.9655
Ep

In [24]:
model.save("model_lstm_sm.keras")
model.save_weights("model_weights_sm.weights.h5")


In [25]:
y_pred_prob = model.predict(x_test,batch_size=8)
y_pred = np.argmax(y_pred_prob, axis=-1)

341/341 [==============================] - 11s 29ms/step


In [26]:
y_pred_labels = [[idx2tag.get(idx) for idx in sentence] for sentence in y_pred]
y_test_labels = [[idx2tag.get(idx) for idx in sentence] for sentence in y_test]

In [27]:
# Step 3: Cross verification - print a few examples
for i in range(5):  # Print the first 5 examples
    print(f"Original Text: {x_test[i]}")  # Here x_test[i] is padded indices, replace with original text if needed
    print(f"True Labels: {y_test_labels[i]}")
    print(f"Predicted Labels: {y_pred_labels[i]}")
    print("-" * 50)

Original Text: [   9   47  181    7  753   11    2  608  736 1006  662 1226    1   16
   86    2  753   23    2  303 1405 1038   13    7  474   91  247   22
    1 1979   32    1  532  241    1    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0  

In [28]:
'''
for i in range(5): 
    print(f"Original Text: {x_test[i]}")  
    print(f"True Labels: {y_test_labels[i]}")
    print(f"Predicted Labels: {y_pred_labels[i]}")
    print("-" * 50)
    '''

'\nfor i in range(5): \n    print(f"Original Text: {x_test[i]}")  \n    print(f"True Labels: {y_test_labels[i]}")\n    print(f"Predicted Labels: {y_pred_labels[i]}")\n    print("-" * 50)\n    '

In [29]:
def compute_accuracy(y_true, y_pred):
    total_labels = 0
    correct_predictions = 0
    for true_sentence, pred_sentence in zip(y_true, y_pred):
        for true_label, pred_label in zip(true_sentence, pred_sentence):
            if true_label != 0: 
                total_labels += 1
                if true_label == pred_label:
                    correct_predictions += 1
    return correct_predictions / total_labels

In [30]:
y_pred = np.argmax(y_pred_prob, axis=-1)
accuracy = compute_accuracy(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9377
